Import the Libs

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


Load the dataset provided

In [4]:
with open('/content/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [5]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [6]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [8]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [9]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [10]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10
3010/3010 [==============================] - 90s 29ms/step - loss: 6.2355 - accuracy: 0.0767
Epoch 2/10
3010/3010 [==============================] - 86s 29ms/step - loss: 5.5244 - accuracy: 0.1240
Epoch 3/10
3010/3010 [==============================] - 86s 28ms/step - loss: 5.1410 - accuracy: 0.1464
Epoch 4/10
3010/3010 [==============================] - 85s 28ms/step - loss: 4.8164 - accuracy: 0.1632
Epoch 5/10
3010/3010 [==============================] - 85s 28ms/step - loss: 4.5187 - accuracy: 0.1813
Epoch 6/10
3010/3010 [==============================] - 87s 29ms/step - loss: 4.2395 - accuracy: 0.1994
Epoch 7/10
3010/3010 [==============================] - 88s 29ms/step - loss: 3.9719 - accuracy: 0.2238
Epoch 8/10
3010/3010 [==============================] - 89s 30ms/step - loss: 3.7205 - accuracy: 0.2533
Epoch 9/10
3010/3010 [==============================] - 89s 30ms/step - loss: 3.4792 - accuracy: 0.2842
Epoch 10/10
3010/3010 [==============================] - 89s 30m

Here you can work with the sentences


In [25]:
s=input("Enter you text here: ")
print("Entered Text::->",s)

seed_text = s
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print("Predicted next words can be::->",seed_text)

Enter you text here: I ate apples because
Entered Text::-> I ate apples because
1/1 [==============================] - 0s 17ms/step
Predicted next words can be::-> I ate apples because i am afraid
